# 01. Environment Setup
## Synthetic Instruction Tuner - Week 1 Day 1-2

This notebook sets up the environment for the entire pipeline:
1. GPU verification (T4 / A100)
2. Library installation
3. Hugging Face authentication
4. Model loading test
5. Project structure setup

**GPU Requirements**:
- **T4 GPU** (Colab Free): 16GB VRAM - Full pipeline supported
- **A100 GPU** (Colab Pro): 40GB VRAM - **Optimized settings available** (2-3x faster)

**Expected runtime**: 10-15 minutes

## 1. GPU Verification

**Note**: This pipeline supports both T4 (free) and A100 (Colab Pro) GPUs.
- **T4**: All notebooks work with default settings
- **A100**: Optimized batch sizes and checkpoint intervals for 2-3x speedup

In [1]:
# Check GPU availability
!nvidia-smi

Tue Dec 23 20:49:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    # Detect GPU type and provide optimization info
    if "A100" in gpu_name:
        print(f"\n✅ A100 GPU detected!")
        print(f"💡 This project is optimized for A100:")
        print(f"   - SFT batch size: 12 (3x faster)")
        print(f"   - DPO batch size: 8 (4x faster)")
        print(f"   - Expected pipeline time: 13-20 hours (vs 33-43h on T4)")
    elif "T4" in gpu_name:
        print(f"\n✅ T4 GPU detected!")
        print(f"💡 Pipeline will use default settings optimized for T4")
        print(f"   - Expected pipeline time: 33-43 hours")
        print(f"   - Consider Colab Pro A100 for 2-3x speedup")
    else:
        print(f"\n⚠️  GPU type: {gpu_name}")
        print(f"💡 This pipeline is optimized for T4 and A100 GPUs")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA version: 12.6
GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 42.47 GB

✅ A100 GPU detected!
💡 This project is optimized for A100:
   - SFT batch size: 12 (3x faster)
   - DPO batch size: 8 (4x faster)
   - Expected pipeline time: 13-20 hours (vs 33-43h on T4)


## 2. Install Required Libraries

In [3]:
# Install core libraries with latest compatible versions
!pip install -q --upgrade transformers>=4.41.0
!pip install -q --upgrade peft>=0.7.0
!pip install -q --upgrade trl>=0.7.4
!pip install -q --upgrade datasets>=2.16.0
!pip install -q --upgrade accelerate>=0.25.0
!pip install -q --upgrade bitsandbytes>=0.41.3
!pip install -q sentencepiece protobuf

# Install evaluation library
!pip install -q lm-eval

# Install utilities
!pip install -q jsonlines huggingface_hub

print("All libraries installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 142.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 8.9 MB/s eta 0:00:00
All libraries installed successfully!


In [4]:
# Verify installations and check versions
import transformers
import peft
import trl
import datasets
import accelerate

print("Installed versions:")
print("=" * 50)
print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"trl: {trl.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"accelerate: {accelerate.__version__}")
print("=" * 50)
print("\n✅ All libraries installed and verified!")

Installed versions:
transformers: 4.57.3
peft: 0.18.0
trl: 0.26.2
datasets: 4.4.2
accelerate: 1.12.0

✅ All libraries installed and verified!


In [5]:
# Verify installations
import transformers
import peft
import trl
import datasets
import accelerate

print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"trl: {trl.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"accelerate: {accelerate.__version__}")

transformers: 4.57.3
peft: 0.18.0
trl: 0.26.2
datasets: 4.4.2
accelerate: 1.12.0


## 3. Google Drive Mount (for Colab)

In [6]:
# Mount Google Drive
from google.colab import drive
from google.colab import userdata
from huggingface_hub import login

drive.mount('/content/drive')

login(token=userdata.get('HF_TOKEN'))

Mounted at /content/drive


In [7]:
# Set project paths
import os

# Change this to your Google Drive path
PROJECT_ROOT = "/content/drive/MyDrive/synthetic-instruction-tuner"

# Create project directories if they don't exist
directories = [
    f"{PROJECT_ROOT}/data/raw",
    f"{PROJECT_ROOT}/data/filtered",
    f"{PROJECT_ROOT}/data/preference",
    f"{PROJECT_ROOT}/models/sft",
    f"{PROJECT_ROOT}/models/dpo",
    f"{PROJECT_ROOT}/evaluation/results",
    f"{PROJECT_ROOT}/evaluation/figures",
]

for dir_path in directories:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created: {dir_path}")

print("\nProject structure ready!")

Created: /content/drive/MyDrive/synthetic-instruction-tuner/data/raw
Created: /content/drive/MyDrive/synthetic-instruction-tuner/data/filtered
Created: /content/drive/MyDrive/synthetic-instruction-tuner/data/preference
Created: /content/drive/MyDrive/synthetic-instruction-tuner/models/sft
Created: /content/drive/MyDrive/synthetic-instruction-tuner/models/dpo
Created: /content/drive/MyDrive/synthetic-instruction-tuner/evaluation/results
Created: /content/drive/MyDrive/synthetic-instruction-tuner/evaluation/figures

Project structure ready!


## 4. Hugging Face Authentication

In [8]:
# Verify login
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"Logged in as: {user_info['name']}")
    print(f"Email: {user_info.get('email', 'N/A')}")
except Exception as e:
    print(f"Login failed: {e}")
    print("Please run the login cell again with a valid token.")

Logged in as: Changyong3
Email: N/A


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Model ID for data generation
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

# 4-bit quantization config for memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

print(f"Loading {MODEL_ID}...")
print("This may take a few minutes on first download.")

Loading meta-llama/Llama-3.1-8B-Instruct...
This may take a few minutes on first download.


In [10]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Tokenizer loaded. Vocab size: 128000


In [11]:
# Load model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)

print(f"Model loaded successfully!")
print(f"Model dtype: {model.dtype}")
print(f"Device: {model.device}")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model loaded successfully!
Model dtype: torch.float16
Device: cuda:0


In [12]:
# Check GPU memory usage
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    print(f"GPU Memory Allocated: {allocated:.2f} GB")
    print(f"GPU Memory Reserved: {reserved:.2f} GB")

GPU Memory Allocated: 5.71 GB
GPU Memory Reserved: 7.62 GB


## 6. Test Inference

In [13]:
# Simple inference test
test_prompt = "What is machine learning?"

messages = [
    {"role": "user", "content": test_prompt}
]

# Format with chat template
input_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Input format:")
print(input_text)
print("="*50)

Input format:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is machine learning?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [14]:
# Generate response
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated response:")
print(response)

Generated response:
system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

user

What is machine learning?assistant

Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable computers to learn from data, make decisions, and improve their performance on a task without being explicitly programmed.

Machine learning involves training a model on a dataset, which allows the model to learn patterns, relationships, and trends in the data. This enables the model to make predictions, classify new data, and optimize its performance on a specific task.

There are several types of machine learning:

1. **Supervised learning**: In this type of learning, the model is trained on labeled data, where the correct output is provided for each input. The model learns to map inputs to outputs based on the labeled data.
2. **Unsupervised learning**: In this type of learning, the model is trained on unlabeled data, and th

## 7. Test Magpie-style Generation

The Magpie method uses only the user template prefix to trigger instruction generation.

In [15]:
# Magpie-style template (only the prefix, no actual prompt)
# This triggers the model to generate an instruction
magpie_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"

print("Magpie template:")
print(repr(magpie_template))

Magpie template:
'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n'


In [16]:
# Generate instruction using Magpie method
inputs = tokenizer(magpie_template, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.9,  # Higher temperature for diversity
        do_sample=True,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("Magpie-generated instruction:")
print(generated)

Magpie-generated instruction:
<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

The following reaction is used to determine the amount of a metal oxide present in a sample of steel:

\[  \text{Cr}_{2}\text{O}_{3}+2\text{Al}\rightarrow\text{Al}_{2}\text{O}_{3}+\text{Cr}\]

In this reaction, 2.00 g of \(\text{Al}\) is used. 0.500 g of \(\text{Cr}_{2}\text{O}_{3}\) is produced in 120 seconds. What is the rate of reaction in \(\text{Cr}_{2}\text{O}_{3}\) per second?

## Step 1: Determine the molar masses of the substances involved.
The molar mass of \(\text{Cr}_{2}\text{O}_{3}\) is approximately 151.99 g/mol, and the molar mass of Al is approximately 26.98 g/mol.

## Step 2: Calculate the number of moles of \(\text{Al}\) used in the reaction.
Number of moles of \(\text{Al}\) = mass of \(\text{Al}\) / molar mass of \(\text{Al}\) = 2


In [17]:
# Parse the generated instruction
def parse_magpie_output(output_text):
    """Extract instruction from Magpie-style generation."""
    # Remove the template prefix
    if "<|start_header_id|>user<|end_header_id|>" in output_text:
        parts = output_text.split("<|start_header_id|>user<|end_header_id|>")
        if len(parts) > 1:
            instruction_part = parts[1]
            # Extract until end of turn or assistant header
            if "<|eot_id|>" in instruction_part:
                instruction = instruction_part.split("<|eot_id|>")[0].strip()
            elif "<|start_header_id|>assistant" in instruction_part:
                instruction = instruction_part.split("<|start_header_id|>assistant")[0].strip()
            else:
                instruction = instruction_part.strip()
            return instruction
    return None

instruction = parse_magpie_output(generated)
print("Parsed instruction:")
print(instruction)

Parsed instruction:
The following reaction is used to determine the amount of a metal oxide present in a sample of steel:

\[  \text{Cr}_{2}\text{O}_{3}+2\text{Al}\rightarrow\text{Al}_{2}\text{O}_{3}+\text{Cr}\]

In this reaction, 2.00 g of \(\text{Al}\) is used. 0.500 g of \(\text{Cr}_{2}\text{O}_{3}\) is produced in 120 seconds. What is the rate of reaction in \(\text{Cr}_{2}\text{O}_{3}\) per second?

## Step 1: Determine the molar masses of the substances involved.
The molar mass of \(\text{Cr}_{2}\text{O}_{3}\) is approximately 151.99 g/mol, and the molar mass of Al is approximately 26.98 g/mol.

## Step 2: Calculate the number of moles of \(\text{Al}\) used in the reaction.
Number of moles of \(\text{Al}\) = mass of \(\text{Al}\) / molar mass of \(\text{Al}\) = 2


## 8. Cleanup

In [18]:
# Free GPU memory
import gc

del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared!")
if torch.cuda.is_available():
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

Memory cleared!
GPU Memory Allocated: 0.01 GB


## 9. Configuration File

In [19]:
# Save project configuration
import json

config = {
    "project_name": "synthetic-instruction-tuner",
    "version": "0.1.0",

    # Paths
    "paths": {
        "project_root": PROJECT_ROOT,
        "data_raw": f"{PROJECT_ROOT}/data/raw",
        "data_filtered": f"{PROJECT_ROOT}/data/filtered",
        "data_preference": f"{PROJECT_ROOT}/data/preference",
        "models_sft": f"{PROJECT_ROOT}/models/sft",
        "models_dpo": f"{PROJECT_ROOT}/models/dpo",
        "evaluation_results": f"{PROJECT_ROOT}/evaluation/results",
        "evaluation_figures": f"{PROJECT_ROOT}/evaluation/figures"
    },

    # Models
    "models": {
        "data_generation": "meta-llama/Llama-3.1-8B-Instruct",
        "fine_tuning_targets": [
            "meta-llama/Llama-3.2-3B",
            "mistralai/Mistral-7B-v0.1",
            "Qwen/Qwen2.5-3B"
        ],
        "reward_model": "OpenAssistant/reward-model-deberta-v3-large-v2"
    },

    # Data generation settings (optimized for Colab free tier)
    "data_generation": {
        "target_raw_samples": 1500,
        "target_filtered_samples": 1000,
        "checkpoint_interval": 20,
        "temperature": 0.9,
        "max_new_tokens": 512
    },

    # Quality filtering settings
    "filtering": {
        "min_words": 20,
        "max_words": 500,
        "jaccard_threshold": 0.8,
        "refusal_keywords": [
            "I'm an AI", "I cannot", "I don't have",
            "As an AI", "I'm not able", "I apologize"
        ]
    },

    # LoRA settings
    "lora": {
        "r": 8,
        "lora_alpha": 16,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
        "lora_dropout": 0.05
    },

    # SFT settings
    "sft": {
        "num_epochs": 3,
        "batch_size": 4,
        "gradient_accumulation_steps": 4,
        "learning_rate": 2e-4,
        "warmup_ratio": 0.03,
        "save_steps": 500
    },

    # DPO settings
    "dpo": {
        "beta": 0.1,
        "num_epochs": 1,
        "batch_size": 2,
        "gradient_accumulation_steps": 8,
        "learning_rate": 5e-5
    },

    # Evaluation benchmarks
    "evaluation": {
        "benchmarks": ["ifeval", "mmlu", "truthfulqa"],
        "batch_size": 4
    }
}

# Save config
config_path = f"{PROJECT_ROOT}/config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"Configuration saved to: {config_path}")

Configuration saved to: /content/drive/MyDrive/synthetic-instruction-tuner/config.json


## ✅ Setup Complete!

### Checklist:
- [x] GPU verified (T4 16GB / A100 40GB)
- [x] Libraries installed
- [x] Google Drive mounted
- [x] Hugging Face authenticated
- [x] Model loading tested
- [x] Magpie-style generation tested
- [x] Configuration saved

### GPU-Specific Notes:
**If using T4**:
- All default settings will work
- Data generation: 3 days (500 samples/day)
- Training time: 6-10h (SFT), 4-6h (DPO)

**If using A100** (Colab Pro):
- Use optimized settings in config.json
- Data generation: Single session (6-8h for 1,500 samples)
- Training time: 2-4h (SFT), 1-2h (DPO)
- Batch sizes automatically optimized

### Next Steps:
1. Proceed to `02_magpie_generation.ipynb` for data generation
2. Follow the checkpoint-based generation strategy for your GPU type